In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

In [2]:
# 11.2 시계열 기초

In [3]:
# pandas에서 찾아볼 수 있는 가장 기본적인 시계열 객체의 종류는 파이썬 문자열이나 datetime 객체로 표현되는 타임스탬프로 색인된 Series다. 

In [5]:
from datetime import datetime

In [6]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]

In [7]:
ts = pd.Series(np.random.randn(6), index=dates)

In [8]:
ts

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

In [9]:
# 내부적으로 보면 이들 datetime 객체는 DatetimeIndex에 들어 있으며 ts 변수의 타입은 TimeSeries다. 

In [10]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [11]:
# 다른 Series와 마찬가지로 서로 다르게 색인된 시계열 객체 간의 산술 연산은 자동으로 날짜에 맞춰진다. 

In [12]:
ts + ts[::2]

2011-01-02   -0.409415
2011-01-05         NaN
2011-01-07   -1.038877
2011-01-08         NaN
2011-01-10    3.931561
2011-01-12         NaN
dtype: float64

In [13]:
# ts[::2]는 ts에서 매 두번째 항목을 선택한다. 

In [14]:
# pandas는 NumPy의 datetime64 자료형을 사용해서 나노초의 정밀도를 가지는 타임스탬프를 저장한다. 

In [15]:
ts.index.dtype

dtype('<M8[ns]')

In [16]:
# DatetimeIndex의 스칼라값은 pandas의 Timestamp 객체다.

In [17]:
stamp = ts.index[0]

In [18]:
stamp

Timestamp('2011-01-02 00:00:00')

In [19]:
# Timestamp는 datetime 객체를 사용하는 어떤 곳에도 대체 사용이 가능하다. 
# 게다가 가능하다면 빈도에 관한 정보도 저장하며 시간대 변환을 하는 방법과 다른 종류의 조작을 하는 방법도 포함하고 있다. 

In [20]:
# 11.2.1 색인, 선택, 부분 선택

In [21]:
# 시계열은 레이블에 기반해서 데이터를 선택하고 인덱싱할 때 pandas.Series와 동일하게 동작한다.

In [22]:
stamp = ts.index[2]

In [23]:
ts[stamp]

-0.5194387150567381

In [24]:
# 해석할 수 있는 날짜를 문자열로 넘겨서 편리하게 사용할 수 있다. 

In [25]:
ts["1/10/2011"]

1.9657805725027142

In [26]:
ts["20110110"]

1.9657805725027142

In [27]:
# 긴 시계열에서는 연을 넘기거나 연, 월만 넘겨서 데이터의 일부 구간만 선택할 수도 있다.

In [28]:
longer_ts = pd.Series(np.random.randn(1000),
                      index=pd.date_range("1/1/2000", periods=1000))

In [29]:
longer_ts

2000-01-01    0.092908
2000-01-02    0.281746
2000-01-03    0.769023
2000-01-04    1.246435
2000-01-05    1.007189
                ...   
2002-09-22    0.930944
2002-09-23   -0.811676
2002-09-24   -1.830156
2002-09-25   -0.138730
2002-09-26    0.334088
Freq: D, Length: 1000, dtype: float64

In [30]:
longer_ts["2001"]

2001-01-01    1.599534
2001-01-02    0.474071
2001-01-03    0.151326
2001-01-04   -0.542173
2001-01-05   -0.475496
                ...   
2001-12-27    0.057874
2001-12-28   -0.433739
2001-12-29    0.092698
2001-12-30   -1.397820
2001-12-31    1.457823
Freq: D, Length: 365, dtype: float64

In [31]:
# 여기서 문자열 "2001"은 연도로 해석되어 해당 기간의 데이터를 선택한다. 
# 월에 대해서도 마찬가지로 선택할 수 있다.

In [32]:
longer_ts["2001-05"]

2001-05-01   -0.622547
2001-05-02    0.936289
2001-05-03    0.750018
2001-05-04   -0.056715
2001-05-05    2.300675
                ...   
2001-05-27    0.235477
2001-05-28    0.111835
2001-05-29   -1.251504
2001-05-30   -2.949343
2001-05-31    0.634634
Freq: D, Length: 31, dtype: float64

In [33]:
# datetime 객체로 데이터를 잘라내는 작업은 일반적인 Series와 동일한 방식으로 할 수 있다. 

In [34]:
ts[datetime(2001, 1, 7):]

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

In [35]:
# 대부분의 시계열 데이터는 연대순으로 정렬되기 때문에 범위를 지정하기 위해 시계열에 포함하지 않고 타임스탬프를 이용해서 Series를 나눌 수 있다.

In [36]:
ts

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

In [37]:
ts["1/6/2011":"1/11/2011"]

2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
dtype: float64

In [39]:
# 앞서와 같이 날짜 문자열이나 datetime 혹은 타임스탬프를 넘길 수 있다. 
# 이런 방식으로 데이터를 나누면 NumPy 배열을 나누는 것처럼 원본 시계열에 대한 뷰를 생성한다는 사실을 기억하자.
# 즉, 데이터 복사가 발생하지 않고 슬라이스에 대한 변경이 원본 데이터에도 반영된다.

In [40]:
# 이와 동일한 인스턴스 메서드로 truncate가 있는데, 이 메서드는 TimeSeries를 두 개의 날짜로 나눈다.

In [41]:
ts.truncate(after="1/9/2011")

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
dtype: float64

In [42]:
# 위 방식은 DataFrame에서도 동일하게 적용되며 로우에 인덱싱된다. 

In [43]:
dates = pd.date_range("1/1/2000", periods=100, freq="W-WED")

In [44]:
long_df = pd.DataFrame(np.random.randn(100, 4),
                       index=dates,
                       columns=["Colorado", "Texas",
                                "New York", "Ohio"])

In [45]:
long_df.loc["5-2001"]

,Colorado,Texas,New York,Ohio
2001-05-02,-0.006045,0.490094,-0.277186,-0.707213
2001-05-09,-0.560107,2.735527,0.927335,1.513906
2001-05-16,0.538600,1.273768,0.667876,-0.969206
2001-05-23,1.676091,-0.817649,0.050188,1.951312
2001-05-30,3.260383,0.963301,1.201206,-1.852001


In [46]:
# 중복된 색인을 갖는 시계열

In [47]:
# 어떤 애플리케이션에서는 여러 데이터가 특정 타임스탬프에 몰려 있는 것을 발견할 수 있다.

In [48]:
dates = pd.DatetimeIndex(["1/1/2000", "1/2/2000", "1/2/2000",
                          "1/2/2000", "1/3/2000"])

In [49]:
dup_ts = pd.Series(np.arange(5), index=dates)

In [50]:
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

In [51]:
# is_unique 속성을 통해 확인해보면 색인이 유일하지 않음을 알 수 있다.

In [52]:
dup_ts.index.is_unique

False

In [53]:
# 이 시계열 데이터를 인덱싱하면 타임스탬프의 중복 여부에 따라 스칼라값이나 슬라이스가 생성된다.

In [54]:
dup_ts["1/3/2000"] # 중복 없음

4

In [55]:
dup_ts["1/2/2000"] # 중복 있음

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int32

In [56]:
# 유일하지 않은 타임스탬프를 가지는 데이터를 집계한다고 해보자.
# 한 가지 방법은 groupby에 level=0(단일 단계 인덱싱)을 넘기는 것이다.

In [61]:
grouped = dup_ts.groupby(level=0)

In [62]:
grouped.mean()

2000-01-01    0.0
2000-01-02    2.0
2000-01-03    4.0
dtype: float64

In [63]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64